In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os
import missingno as msno

In [63]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



In [67]:
# Carregando DataFrames das edições do censo da educação superior.
censo_12 = 'censo_2012.csv'
file_ies = 'censo_ies/MICRODADOS_CADASTRO_IES_2012.CSV'

In [68]:
pd.set_option('display.max_columns', 200)

In [69]:
pd.set_option('display.max_rows', 500)

In [70]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_12, 'rb') as f1:
    raw_data_12 = f1.read(1000000)

In [71]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data_12)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

codificação detectada: ISO-8859-1 (confiança: 0.73%)


In [72]:
df_12 = pd.read_csv(censo_12, sep=';', encoding=cod_23, low_memory=False, decimal=',')
df_12.shape

(63855, 194)

In [73]:
cols = ['CO_IES', 'NO_REGIAO_IES', 'CO_REGIAO_IES', 'NO_UF_IES', 'SG_UF_IES', 'CO_UF_IES', 'NO_MUNICIPIO_IES', 'CO_MUNICIPIO_IES', 'IN_CAPITAL_IES']

In [74]:
ies_12 = pd.read_csv(file_ies, sep=';', encoding=cod_23, low_memory=False, decimal=';', usecols=cols)
ies_12.shape

(2416, 9)

In [75]:
(df_12.isna().mean()*100).round(2)

NU_ANO_CENSO                     0.00
CO_IES                           0.00
NO_CURSO                         0.00
CO_CURSO                         0.00
TP_GRAU_ACADEMICO                0.52
IN_GRATUITO                      0.00
TP_MODALIDADE_ENSINO             0.00
TP_NIVEL_ACADEMICO               0.00
NO_CINE_AREA_GERAL               0.00
CO_CINE_AREA_GERAL               0.00
CO_CINE_AREA_ESPECIFICA          0.00
NO_CINE_AREA_ESPECIFICA          0.00
CO_CINE_AREA_DETALHADA           0.00
NO_CINE_AREA_DETALHADA           0.00
NO_REGIAO                        1.80
CO_REGIAO                        1.80
NO_UF                            1.80
SG_UF                            1.80
CO_UF                            1.80
NO_MUNICIPIO                     1.80
CO_MUNICIPIO                     1.80
IN_CAPITAL                       1.80
TP_DIMENSAO                      0.00
TP_ORGANIZACAO_ACADEMICA         0.00
TP_REDE                          0.00
TP_CATEGORIA_ADMINISTRATIVA      0.00
QT_CURSO    

In [76]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_11.columns if col.startswith('QT_')]
len(colunas_quantidade)

168

In [77]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
for col in colunas_quantidade:
    df_12[col] = df_12[col].fillna(0)

# Solução alternativa usando .loc
# df_11.loc[:,colunas_quantidade] = df_11[colunas_quantidade].fillna(0)

Preenchendo os valores ausentes da coluna **'TP_GRAU_ACADEMICO'**

In [78]:
# 1. vou criar uma série (mascara booleana) (True onde é nan)
mask_na = df_12['TP_GRAU_ACADEMICO'].isna()

In [79]:
# 2. Aplicar a mascara boleanda para selecionar apenas as linhas com valor True
g_acadna = df_12[mask_na]

Para começar a preencher os valores ausentes da coluna 'TP_GRAU_ACADEMICO', algumas decisões de análise precisarão ser tomadas.

Para reduzir o número de valores ausentes na coluna 'TP_NIVEL_ACADEMICO'

In [80]:
# 3. Verificando os valores únicos da variável 'TP_NIVEL_ACADEMICO'
g_acadna['TP_NIVEL_ACADEMICO'].unique()

array([2, 1], dtype=int64)

In [81]:
# 4. Criando uma mascara booleanda apenas com os cursos Sequencial de formação especifica.
mask_cs = g_acadna['TP_NIVEL_ACADEMICO'] == 2 # Sequencial de formação específica

In [82]:
# 5. Aplicando a mascara booleana para para selecionar apenas as linha com valor True da variável 'TP_NIVEL_ACADEMICO'
c_fesp = g_acadna[mask_cs]

In [83]:
c_fesp.shape

(191, 194)

In [84]:
# 6. Verificando as incidências de valores da variável 'NO_CINE_AREA_GERAL'
c_fesp['NO_CINE_AREA_GERAL'].value_counts()

NO_CINE_AREA_GERAL
Negócios, administração e direito                             137
Computação e Tecnologias da Informação e Comunicação (TIC)     21
Serviços                                                       12
Artes e humanidades                                            11
Engenharia, produção e construção                               7
Saúde e bem-estar                                               2
Ciências naturais, matemática e estatística                     1
Name: count, dtype: int64

In [85]:
# 7. Criando uma cópia do DataFrame para realizar as imputações
df_2012 = df_12.copy()

In [86]:
# 7.1 Checando as dimensões do novo DF
df_2012.shape

(63855, 194)

In [87]:
# 7.2 Verificando os valores ausentes na variável alvo.
df_2012['TP_GRAU_ACADEMICO'].isna().sum()

335

In [89]:
# 7.3 Criando uma validação condicional utilizando as duas mascaras criadas anteriormente e atribuindo um novo valor para a variável alvo.
condition = (mask_na & mask_cs)
df_2012.loc[condition, 'TP_GRAU_ACADEMICO'] = 5 #  Sequencial de formação específica

In [90]:
# 7.4 Conferindo a redução no número valores ausentes na variável alvo
df_2012['TP_GRAU_ACADEMICO'].isna().sum()

144

In [91]:
# 8. Criando uma nova mascara de seleção dos valores ausentes remanecentes na variável 'TP_GRAU_ACADEMICO'
mask1_na = df_2012['TP_GRAU_ACADEMICO'].isna()

In [92]:
# 8.1 Criação de um novo df filtrado pela mascara criada anteriormente
g_acadna1 = df_2012[mask1_na]

In [93]:
# 8.2 Verificação da dimensão do df olhando apenas para a quantidade de linhas
g_acadna1.shape[0]

144

In [94]:
# 8.3 Verificando algumas amostras de linhas do DataFrame para verificar a insidência de algum padrão
g_acadna1.sample(5)

,NU_ANO_CENSO,CO_IES,NO_CURSO,CO_CURSO,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,NO_CINE_AREA_GERAL,CO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_REDE,TP_CATEGORIA_ADMINISTRATIVA,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PRETA.1,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVSOCIAL_RF.1,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
11030,2012,8,ABI - CIÊNCIAS SOCIAIS,5000602,NaN,1,1,1,Programas básicos,0,1,Programas básicos,11,Programas básicos,Sudeste,3.0,Minas Gerais,MG,31.0,Viçosa,3171303.0,0.0,1,1,1,1,0.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,49.0,30.0,19.0,0.0,49.0,0.0,8.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,8.0,5.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,48.0,33.0,16.0,17.0,0.0,33.0,0.0,24.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [95]:
# 8.4 Analisando algumas estatísticas das variáveis não númericas do DataFrame
g_acadna1.describe(include = 'object')

,NO_CURSO,NO_CINE_AREA_GERAL,NO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_DETALHADA,NO_REGIAO,NO_UF,SG_UF,NO_MUNICIPIO
count,144,144,144,144,144,144,144,144
unique,59,5,5,5,5,14,14,36
top,ABI - CIÊNCIAS BIOLÓGICAS,Programas básicos,Programas básicos,Programas básicos,Sudeste,São Paulo,SP,Rio de Janeiro
freq,14,140,140,140,113,56,56,15


In [96]:
#  8.4.1 Verificando as regiões do Brasil que não preencheram a variável ´TP_GRAU_ACADEMICO´ para o cursos diferentes de ...
# 'Sequencial de formação especifica"
g_acadna1['NO_REGIAO'].value_counts()

NO_REGIAO
Sudeste         113
Sul              16
Centro-Oeste      9
Norte             3
Nordeste          3
Name: count, dtype: int64

In [97]:
# 8.4.2 Verificando insidência de valores ausentes por tipo de Rede (Pública e Privada)
g_acadna1['TP_REDE'].value_counts()

TP_REDE
1    136
2      8
Name: count, dtype: int64

In [98]:
# 8.4.3 Visualizando a ocorrência dos cursos para determinar quais são bacharelados por definição e quais não são.
g_acadna1['NO_CURSO'].value_counts()

NO_CURSO
ABI - CIÊNCIAS BIOLÓGICAS                       14
ABI - QUÍMICA                                   10
ABI - CIÊNCIAS SOCIAIS                          10
ABI - GEOGRAFIA                                  8
ABI - FÍSICA                                     8
ABI - MATEMÁTICA                                 6
ABI - HISTÓRIA                                   6
ABI - FILOSOFIA                                  6
ABI - LETRAS                                     5
ABI - ENGENHARIA                                 5
ABI - ARTES VISUAIS                              4
ABI - EDUCAÇÃO FÍSICA                            4
ABI - PSICOLOGIA                                 3
ABI - LETRAS FRANCÊS                             2
MATEMÁTICA - LICENCIATURA OU BACHARELADO         2
FÍSICA - LICENCIATURA OU BACHARELADO             2
ABI - ARTES CÊNICAS                              2
ABI - LETRAS ITALIANO                            2
ABI - LETRAS INGLÊS                              2
ABI - CIÊNCIAS DA NATU

In [101]:
g_acadna1['NO_CURSO'].unique()

array(['ABI - GESTÃO TERRITORIAL', 'ABI - LICENCIATURA INTERCULTURAL',
       'ABI - FORMAÇÃO INTERDISCIPLINAR',
       'ABI - ENGENHARIA DE ENERGIAS E MEIO AMBIENTE', 'ABI - ENGENHARIA',
       'ABI - CIÊNCIAS BIOLÓGICAS', 'ABI - CIÊNCIAS SOCIAIS',
       'ABI - FILOSOFIA', 'ABI - FÍSICA', 'ABI - GEOGRAFIA',
       'ABI - QUÍMICA', 'ABI - TEATRO', 'ABI - HISTÓRIA', 'ABI - LETRAS',
       'ABI - DANÇA', 'ABI - EDUCAÇÃO FÍSICA', 'ABI - MATEMÁTICA',
       'ABI - CIÊNCIAS DA NATUREZA', 'ABI - LETRAS FRANCÊS',
       'ABI - LETRAS GREGO', 'ABI - LETRAS ITALIANO', 'ABI - PSICOLOGIA',
       'ABI - COMUNICAÇÃO SOCIAL', 'ABI - LETRAS - PORTUGUÊS - ARABE',
       'ABI - LETRAS - PORTUGUÊS - ESPANHOL',
       'ABI - LETRAS - PORTUGUÊS - FRANCES',
       'ABI - LETRAS - PORTUGUÊS - GREGO',
       'ABI - LETRAS - PORTUGUÊS - HEBRAICO',
       'ABI - LETRAS - PORTUGUÊS - INGLES',
       'ABI - LETRAS - PORTUGUÊS - ITALIANO',
       'ABI - LETRAS - PORTUGUÊS - JAPONES',
       'ABI - LETRAS - PORT

In [104]:
# 8.4.4 Criando uma mascara classificando os cursos de licenciatura e bacharelado
no_licenciaturas = [
    'ABI - GESTÃO TERRITORIAL',
    'ABI - FORMAÇÃO INTERDISCIPLINAR', # Geralmente leva a um Bacharelado Interdisciplinar.
    'ABI - ENGENHARIA DE ENERGIAS E MEIO AMBIENTE',
    'ABI - ENGENHARIA',
    'ABI - PSICOLOGIA', # Foco clínico/pesquisa (Bacharelado).
    'ABI - COMUNICAÇÃO SOCIAL',
    'ABI - DESIGN',
    'ABI - CONSTRUÇÃO CIVIL',
    'ABI - TECNOLOGIA AMBIENTAL',
    'ABI - ECONOMIA EMPRESARIAL E CONTROLADORIA',
    'ABI - MEDICINA', # Profissionalizante.
    'ABI - SISTEMAS DE INFORMAÇÃO',
    'ABI - ENGENHARIA MECÂNICA',
    'ABI - ENGENHARIA DA MOBILIDADE'
]
len(no_licenciaturas)

14

In [105]:
# 8.4.4.2 Criando a condição lógica para definir a atribuição de valores para as outras variáveis
lic_bac = [
    'ABI - LICENCIATURA INTERCULTURAL',
    'ABI - CIÊNCIAS BIOLÓGICAS',
    'ABI - CIÊNCIAS SOCIAIS',
    'ABI - FILOSOFIA',
    'ABI - FÍSICA',
    'ABI - GEOGRAFIA',
    'ABI - QUÍMICA',
    'ABI - TEATRO',
    'ABI - HISTÓRIA',
    'ABI - LETRAS',
    'ABI - DANÇA',
    'ABI - EDUCAÇÃO FÍSICA', # Oferece L (Escola) e B (Clube/Academia)
    'ABI - MATEMÁTICA',
    'ABI - CIÊNCIAS DA NATUREZA',
    'ABI - LETRAS FRANCÊS',
    'ABI - LETRAS GREGO',
    'ABI - LETRAS ITALIANO',
    'ABI - LETRAS - PORTUGUÊS - ARABE',
    'ABI - LETRAS - PORTUGUÊS - ESPANHOL',
    'ABI - LETRAS - PORTUGUÊS - FRANCES',
    'ABI - LETRAS - PORTUGUÊS - GREGO',
    'ABI - LETRAS - PORTUGUÊS - HEBRAICO',
    'ABI - LETRAS - PORTUGUÊS - INGLES',
    'ABI - LETRAS - PORTUGUÊS - ITALIANO',
    'ABI - LETRAS - PORTUGUÊS - JAPONES',
    'ABI - LETRAS - PORTUGUÊS - LATIM',
    'ABI - LETRAS - PORTUGUÊS - RUSSO',
    'ABI - LETRAS PORTUGUÊS - ALEMÃO',
    'ABI - LITERATURAS DE LÍNGUA PORTUGUESA',
    'ABI - ARTES VISUAIS',
    'ABI - FÍSICA / MATEMÁTICA APLICADA',
    'ABI - MÚSICA',
    'FILOSOFIA - LICENCIATURA OU BACHARELADO',
    'FÍSICA - LICENCIATURA OU BACHARELADO',
    'MATEMÁTICA - LICENCIATURA OU BACHARELADO',
    'ABI - CIÊNCIAS EXATAS',
    'ABI - ARTES CÊNICAS',
    'ABI - EDUCAÇÃO FÍSICA E ESPORTE',
    'ABI - MATEMÁTICA APLICADA',
    'ABI - MATEMÁTICA APLICADA E COMPUTACIONAL',
    'ABI - LETRAS INGLÊS',
    'ABI - LETRAS ALEMÂO',
    'ABI - LETRAS ESPANHOL',
    'ABI - LETRAS LÍNGUA PORTUGUESA',
    'ABI - EDUCAÇÃO INTERCULTURAL'
]

In [ ]:
# 8.4.4.1 Criando a condição lógica para definir a atribuição de valores
mask2 = df_2012['NO_CURSO'].isin(no_licenciaturas)
mask3 = df_2012['TP_GRAU_ACADEMICO'].isna()
mask4 = df_2012['NO_CURSO'].isin(lic_bac)
condiction1 = (mask2 & mask3)
condiction2 = (mask4 & mask3)

In [ ]:
# Conferindo número de atribuições da condiction1
#condiction1.sum()

In [ ]:
#df_2011['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 8.4.4.3 Atribuição para os registros que atendem a condição de não licenciaturas
#df_2011.loc[condiction1, 'TP_GRAU_ACADEMICO'] = 1 # Bacharelado

In [ ]:
#df_2011['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 8.4.4.4 Atribuição para os registros que atendem a condição de licenciaturas
#df_2011.loc[condiction2, 'TP_GRAU_ACADEMICO'] = 2 # licenciatura

In [ ]:
# Verificando o número de atribuições da condiction2
#condiction2.sum()

In [ ]:
# 9. Verificação da incidência de valores nulos na variável 'TP_GRAU_ACADEMICO'
#df_2011['TP_GRAU_ACADEMICO'].isna().sum()

**Tratamento das demais variáveis com valores ausentes**

['NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL']

In [18]:
(df_2011.isna().mean()*100).round(2)

NU_ANO_CENSO                    0.00
CO_IES                          0.00
NO_CURSO                        0.00
CO_CURSO                        0.00
TP_GRAU_ACADEMICO               0.00
IN_GRATUITO                     0.00
TP_MODALIDADE_ENSINO            0.00
TP_NIVEL_ACADEMICO              0.00
NO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_ESPECIFICA         0.00
NO_CINE_AREA_ESPECIFICA         0.00
CO_CINE_AREA_DETALHADA          0.00
NO_CINE_AREA_DETALHADA          0.00
NO_REGIAO                       1.81
CO_REGIAO                       1.81
NO_UF                           1.81
SG_UF                           1.81
CO_UF                           1.81
NO_MUNICIPIO                    1.81
CO_MUNICIPIO                    1.81
IN_CAPITAL                      1.81
TP_DIMENSAO                     0.00
TP_ORGANIZACAO_ACADEMICA        0.00
TP_REDE                         0.00
TP_CATEGORIA_ADMINISTRATIVA     0.00
QT_CURSO                        0.00
Q

Aplicação da função .map() para otimização **'NO_REGIAO'**

In [19]:
# 1. Verificando as dimensões do df lookup (Capastro_ies - fonte de mapeamento)
ies_11.shape

(2365, 9)

In [20]:
# 1.2 Verificando se existe dados duplicados na variável chave do df lookup
ies_11['CO_IES'].duplicated().sum()

0

In [23]:
# 2. Criação da série de mapeamento (dicionário de lookup) index= CO_IES (chave) Values = 'NO_REGIAO'
map_cat1 = ies_11.set_index('CO_IES')['NO_REGIAO_IES']
# O método .drop_duplicates() é essencial para garantir que a chave (CO_IES) seja única, evitando que o .map() falhe ou retorne valores incorretos.
#map_cat1 = map_cat1.drop_duplicates() # no caso atual, não é necessário pois a os registro do df ies_10 são 1 para 1.

In [24]:
map_cat1.head()

CO_IES
1    Centro-Oeste
2    Centro-Oeste
3        Nordeste
4           Norte
5        Nordeste
Name: NO_REGIAO_IES, dtype: object

In [25]:
# Gerando uma nova serie com os valores do mapeamento
cat_map = df_2011['CO_IES'].map(map_cat1)

In [26]:
df_2011['NO_REGIAO'] = df_2011['NO_REGIAO'].fillna(cat_map)

In [27]:
df_2011['NO_REGIAO'].isna().sum()

0

Aplicando a função na variável **'CO_REGIAO'**

In [29]:
# 1. Criação da série de mapeamentos (lookup)
map_cat2 = ies_11.set_index('CO_IES')['CO_REGIAO_IES']

In [30]:
# 2. Gerando série com os valores do mapeamento
cat_map2 = df_2011['CO_IES'].map(map_cat2) 

In [31]:
# 3. Preechimento de valores com .fillna()
df_2011['CO_REGIAO'] = df_2011['CO_REGIAO'].fillna(cat_map2)

In [32]:
# 4. Validando a aplicação
df_2011['CO_REGIAO'].isna().sum()

0

Aplicando a função na variável **'NO_UF'**

In [33]:
# 1. Criação da série de mapeamento (lookup)
map_cat3 = ies_11.set_index('CO_IES')['NO_UF_IES']

In [34]:
# 2. Gerando série com os valores do mapeamento
cat_map3 = df_2011['CO_IES'].map(map_cat3)

In [35]:
# 3. Preenchimento de valores com .fillna()
df_2011['NO_UF'] = df_2011['NO_UF'].fillna(cat_map3)

In [36]:
# 4. Validação da aplicação
df_2011['NO_UF'].isna().sum()

0

Aplicando a função na variável **'SG_UF'**

In [37]:
# 1. Criação da série de mapeamento (lookup)
map_cat4 = ies_11.set_index('CO_IES')['SG_UF_IES']

In [38]:
# 2. Gerando série com os valores do mapeamento
cat_map4 = df_2011['CO_IES'].map(map_cat4)

In [39]:
# 3. Preenchimento de valores com .fillna()
df_2011['SG_UF'] = df_2011['SG_UF'].fillna(cat_map4)

In [40]:
# 4. Validação da aplicação
df_2011['SG_UF'].isna().sum()

0

Aplicando a função na variável **'CO_UF'**

In [41]:
df_2011.CO_UF.isna().sum()

1047

In [42]:
# 1. Criação de série de mapeamento (lookup)
map_cat5 = ies_11.set_index('CO_IES')['CO_UF_IES']

In [43]:
# 2. Gerando série com os valores do mapemento
cat_map5 = df_2011['CO_IES'].map(map_cat5)

In [44]:
# 3. Preechimento de valores com .fillna()
df_2011['CO_UF'] = df_2011['CO_UF'].fillna(cat_map5)

In [45]:
# 4. Validação da aplicação
df_2011.CO_UF.isna().sum()

0

Aplicando a função na variável **'NO_MUNICIPIO'**

In [46]:
df_2011.NO_MUNICIPIO.isna().sum()

1047

In [47]:
# 1. Criação de série de mapeamento (lookup)
map_cat6 = ies_11.set_index('CO_IES')['NO_MUNICIPIO_IES']

In [48]:
# 2. Gerando série com os valores do mapeamento
cat_map6 = df_2011['CO_IES'].map(map_cat6)

In [49]:
# 3. Preenchimento de valores com fillna()
df_2011['NO_MUNICIPIO'] = df_2011['NO_MUNICIPIO'].fillna(cat_map6)

In [50]:
# 4. Validação
df_2011['NO_MUNICIPIO'].isna().sum()

0

Aplicando a função na variável **'CO_MUNICIPIO'**

In [ ]:
df_2011.CO_MUNICIPIO.isna().sum()

In [51]:
# 1. Criação de série de mapeamento (lookup)
map_cat7 = ies_11.set_index('CO_IES')['CO_MUNICIPIO_IES']

In [52]:
# 2. Gerando série com os valores do mapeamento
cat_map7 = df_2011['CO_IES'].map(map_cat7)

In [53]:
# 3. Preenchimento de valores com fillna()
df_2011['CO_MUNICIPIO'] = df_2011['CO_MUNICIPIO'].fillna(cat_map7)

In [54]:
# 4. Validação
df_2011['CO_MUNICIPIO'].isna().sum()

0

Aplicando a função na variável **'CO_MUNICIPIO'**

In [55]:
df_2011.IN_CAPITAL.isna().sum()

1047

In [56]:
# 1. Criação de série de mapeamento (lookup)
map_cat8 = ies_11.set_index('CO_IES')['IN_CAPITAL_IES']

In [57]:
# 2. Gerando série com os valores do mapeamento
cat_map8 = df_2011['CO_IES'].map(map_cat8)

In [58]:
# 3. Preenchimento de valores com fillna()
df_2011['IN_CAPITAL'] = df_2011['IN_CAPITAL'].fillna(cat_map8)

In [59]:
# 4. Validação
df_2011['IN_CAPITAL'].isna().sum()

0

In [60]:
# Verificando demais variáveis com valores nulos
(df_2011.isna().mean()*100).round(2)

NU_ANO_CENSO                    0.0
CO_IES                          0.0
NO_CURSO                        0.0
CO_CURSO                        0.0
TP_GRAU_ACADEMICO               0.0
IN_GRATUITO                     0.0
TP_MODALIDADE_ENSINO            0.0
TP_NIVEL_ACADEMICO              0.0
NO_CINE_AREA_GERAL              0.0
CO_CINE_AREA_GERAL              0.0
CO_CINE_AREA_ESPECIFICA         0.0
NO_CINE_AREA_ESPECIFICA         0.0
CO_CINE_AREA_DETALHADA          0.0
NO_CINE_AREA_DETALHADA          0.0
NO_REGIAO                       0.0
CO_REGIAO                       0.0
NO_UF                           0.0
SG_UF                           0.0
CO_UF                           0.0
NO_MUNICIPIO                    0.0
CO_MUNICIPIO                    0.0
IN_CAPITAL                      0.0
TP_DIMENSAO                     0.0
TP_ORGANIZACAO_ACADEMICA        0.0
TP_REDE                         0.0
TP_CATEGORIA_ADMINISTRATIVA     0.0
QT_CURSO                        0.0
QT_VG_TOTAL                 

In [61]:
# Salvando em .csv
df_2011.to_csv('censo_2011.csv', sep=';', encoding=cod_23, index=False)

### Fim